<a href="https://colab.research.google.com/github/mgonzal1/nma_wombat_connectivy_project/blob/matrices/connectivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#@title Data retrieval
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)


#@title Data loading
import numpy as np

alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx. 
dat = alldat[11]
print(dat.keys())

dict_keys(['spks', 'wheel', 'pupil', 'lfp', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'brain_area_lfp', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'waveform_w', 'waveform_u', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'lfp_passive', 'pupil_passive', 'wheel_passive'])


In [13]:
# groupings of brain regions
# regions = ["vis ctx", "thal", "hipp", "other ctx", "midbrain", "basal ganglia", "cortical subplate", "other"]
# brain_groups = [["VISa", "VISam", "VISl", "VISp", "VISpm", "VISrl"], # visual cortex
#                 ["CL", "LD", "LGd", "LH", "LP", "MD", "MG", "PO", "POL", "PT", "RT", "SPF", "TH", "VAL", "VPL", "VPM"], # thalamus
#                 ["CA", "CA1", "CA2", "CA3", "DG", "SUB", "POST"], # hippocampal
#                 ["ACA", "AUD", "COA", "DP", "ILA", "MOp", "MOs", "OLF", "ORB", "ORBm", "PIR", "PL", "SSp", "SSs", "RSP"," TT"], # non-visual cortex
#                 ["APN", "IC", "MB", "MRN", "NB", "PAG", "RN", "SCs", "SCm", "SCig", "SCsg", "ZI"], # midbrain
#                 ["ACB", "CP", "GPe", "LS", "LSc", "LSr", "MS", "OT", "SNr", "SI"], # basal ganglia 
#                 ["BLA", "BMA", "EP", "EPd", "MEA"]] # cortical subplate

def get_trainning_set():
  """
  Get a subset of alldat for trainning purposes.

  Returns:
    map: dat['spks']: neurons by trials by time bins. Time bin = 10ms.
         dat['brain_area']: brain area for each neuron recorded.
         dat['contrast_right']: contrast level for the right stimulus, which is always contralateral to the recorded brain areas.
         dat['contrast_left']: contrast level for left stimulus.
         dat['response']: which side the response was (-1, 0, 1). Choices for the right stimulus are -1.
         dat['response_times']: when the response was registered, which has to be after the go cue. The mouse can turn the wheel before the go cue (and always does!).
         dat['wheel']: exact position of the wheel that the mice uses to make a response, binned at 10ms.
         dat['pupil']: pupil area (noisy, because pupil is very small).
         dat['lfp']: recording of the local field potential in each brain area from this experiment, binned at 10ms.
         dat['brain_area_lfp']: brain area names for the LFP channels.
  """
  # Functions to split data and get a subset. If condition of areas  
  trainning_set = alldat
  return trainning_set



def get_validation_set():
  """
  Get a subset of alldat for validation purposes. This should be ~5%-10% of all the data.

  Returns:
    map: dat['spks']: neurons by trials by time bins. Time bin = 10ms.
         dat['brain_area']: brain area for each neuron recorded.
         dat['contrast_right']: contrast level for the right stimulus, which is always contralateral to the recorded brain areas.
         dat['contrast_left']: contrast level for left stimulus.
         dat['response']: which side the response was (-1, 0, 1). Choices for the right stimulus are -1.
         dat['response_times']: when the response was registered, which has to be after the go cue. The mouse can turn the wheel before the go cue (and always does!).
         dat['wheel']: exact position of the wheel that the mice uses to make a response, binned at 10ms.
         dat['pupil']: pupil area (noisy, because pupil is very small).
         dat['lfp']: recording of the local field potential in each brain area from this experiment, binned at 10ms.
         dat['brain_area_lfp']: brain area names for the LFP channels.
  """
  # Functions to split data and get a subset 
  validation_set = alldat
  return validation_set

# def get_visual_spikes(data_set):
#   return

#def get_motor_spikes(trainning_set):

  # movement related brain areas
  # ["VISp", "VISl", "VISpm","VISam","VISa"] # "vis ctx"
  # ["ACA", "MOs", "PL", "MOp", "SSp"] # non-visual cortex
  # ["CP", "SNr"] # basal ganglia 
  # ["LD", "PO"] # thalamus
  # ["SCm","MRN", "APN", "ZI"]   # midbrain

  # Logistic regression module matrices
  # D = M(A3)
  # A3 =  [nNeuronsx1]
  # M = [nTrialsxnNeurons]

  braina = ["VISp", "VISl", "VISpm","VISam","VISa","ACA", "MOs", "PL", "MOp", "SSp", "CP", "SNr", "LD", "PO","SCm","MRN", "APN", "ZI"]   
  sesion = 3
  dat = alldat[3]
  print(dat['brain_area'])

def get_sesion_data():
# ------------------------------------------------------------------------------

  # # Este código sólo guarda neuronas para un área en particular, se necesita hacer 
  # # esto mismo para cada área

  nSesion = alldat.shape    # Numero de sesiones  
  regionTarget = 'CA1'      # Región de interés

  # # Primero cuenta el número de neuronas de la region buscada y calcula la sesión
  # # que haya tenido el mayor número de  ensayos
  nNeurons,nTrials = [0,0]

  for i,dat in enumerate(alldat):
  
    # Índice de la region
    idx_reg = np.where(dat['brain_area'] == regionTarget)[0]

    # Tamaño de la matriz de espigas,shape_1,shape_2,shape_3
    shp0,shp1,shp2 = dat['spks'][idx_reg,:,:].shape

    if shp0 > 0:    # Si la region existe en esa sesión entonces cuenta
      nNeurons = nNeurons + shp0
      nTrials  = np.max([nTrials, shp1])
      ntBins   = shp2

  # Genera una matriz de NaN con nNeuronas x nTrials x nTimeBins
  allSpks    = np.empty([nNeurons,nTrials,ntBins],dtype=np.float32)
  allSpks[:] = np.NaN

  # Rellena la matriz allSpks con las 'spks' de cada sesión
  cont = 0

  for i,dat in enumerate(alldat):
    # Índice de la region
    idx_reg = np.where(dat['brain_area'] == regionTarget)[0]

    shp0,shp1,shp2 = dat['spks'][idx_reg,:,:].shape

    if shp0 > 0:    # Si la region existe entonces guarda neuronas en allSpks
      allSpks[cont:cont+shp0,0:shp1,0:shp2] = dat['spks'][idx_reg,:,:]

    cont = cont + shp0


  return allSpks


# ------------------------------------------------------------------------------

# def get_stimulus(data_set):
#   return

# def get_decision(data_set):
#   return
